In [22]:
import pandas as pd
import os
import FinanceDataReader as fdr

code_data = pd.read_csv('raw_data.csv',index_col=0)
code_data['종목코드'] = code_data['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list=code_data.values.tolist()

In [23]:
code_data.head(3)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,000210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,004840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시


# 과제 1

In [24]:
# dictionary 저장 후 파일 쓰기
code2name_dic = {}

for row in stock_list:
    name, code, date = row[0], row[1], row[4]
    if date <= '2018-01-01': 
        code2name_dic[code] = name 
        
        
        
OF = open('assignment1_multi_processing.txt', 'w', encoding='UTF-8')

for cd in code2name_dic: 
    OF.write(f'{cd}\t{code2name_dic[cd]}\n')
OF.close()

In [25]:
code2name_dic

{'000210': 'DL',
 '004840': 'DRB동일',
 '155660': 'DSR',
 '078930': 'GS',
 '001250': 'GS글로벌',
 '039570': 'HDC랩스',
 '011200': 'HMM',
 '092220': 'KEC',
 '016380': 'KG동부제철',
 '001390': 'KG케미칼',
 '058860': 'KTis',
 '011070': 'LG이노텍',
 '037560': 'LG헬로비전',
 '108670': 'LX하우시스',
 '010060': 'OCI',
 '096770': 'SK이노베이션',
 '011810': 'STX',
 '024070': 'WISCOM',
 '011420': '갤럭시아에스엠',
 '002100': '경농',
 '012320': '경동인베스트',
 '007690': '국도화학',
 '005320': '국동',
 '014530': '극동유화',
 '128820': '대성산업',
 '047040': '대우건설',
 '003090': '대웅',
 '145720': '덴티움',
 '027740': '마니커',
 '008560': '메리츠증권',
 '017180': '명문제약',
 '204210': '모두투어리츠',
 '003850': '보령제약',
 '001270': '부국증권',
 '006400': '삼성SDI',
 '028050': '삼성엔지니어링',
 '068290': '삼성출판사',
 '003720': '삼영화학공업',
 '010960': '삼호개발',
 '001290': '상상인증권',
 '136490': '선진',
 '002420': '세기상사',
 '004490': '세방전지',
 '012600': '센트럴인사이트',
 '055550': '신한지주',
 '003620': '쌍용자동차',
 '078520': '에이블씨엔씨',
 '012280': '영화금속',
 '007310': '오뚜기',
 '271560': '오리온',
 '105840': '우진',
 '008600': '윌비스'

# 과제2

In [26]:
from tqdm import tqdm

In [27]:
# 예시데이터
start_date = '20180102'
end_date = '20201231'
stock = fdr.DataReader(cd, start = start_date, end = end_date)
stock

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-01-02,9890,10000,8800,10000,8210,-0.019608
2018-01-03,9800,9800,9700,9700,800,-0.030000
2018-01-04,9410,9410,9410,9410,200,-0.029897
2018-01-05,9120,10000,9120,10000,140,0.062699
2018-01-08,10000,10000,9850,10000,495,0.000000
...,...,...,...,...,...,...
2020-12-23,4990,4990,4100,4540,272,0.031818
2020-12-24,4595,4595,4005,4515,705,-0.005507
2020-12-28,4985,4985,4065,4640,10,0.027685


In [1]:
import time, os
from multiprocessing import Pool
import multiprocessing


def make_com_code_date(com_code):
    stock_df = fdr.DataReader(com_code, start='2021-01-01',end='2021-07-01')
    date_lst = stock_df.index
    stock_lst = stock_df.values.tolist()
    hot_date_lst = []
    
    for date, row in zip(date_lst, stock_lst):
        date = date.date().strftime("%Y%m%d")
        close, volume = row[3], row[4]
        trading_value = close * volume
    
    
        if trading_value >= 100000000000: # 1000억 이상 10,000,000,000
            hot_date_lst.append(date)


    return [com_code, hot_date_lst]

### multi_processing

In [29]:
start_time=time.time()
com_code_lst = list(code2name_dic.keys())


# set_args=(result_dic,com_code_lst)

if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=20)
    result = pool.map( make_com_code_date, com_code_lst)
    pool.close()
    pool.join()
    
print("---%s seconds ---" % (time.time() - start_time))

---10.562158823013306 seconds ---


In [30]:
OF = open('assignment2_multi_processing.txt','w')
for code, date_lst in result:
    if len(date_lst) != 0:
        for date in date_lst:
            OF.write(f'{code}\t{date}\n')
            
OF.close()

# 과제3

In [36]:
result[0] # [[회사명,[날짜,날짜,날짜, ...]],] 형식의 리스트 (1000억 이상)

['000210', ['20210125', '20210204', '20210512']]

In [37]:
len(result)

1999

In [41]:
def make_merge_stock_data(result_in_assignment2):
    
    com_code, date_lst = result_in_assignment2
    stock_df = fdr.DataReader(com_code, start='2021-01-01',end='2021-07-02').reset_index()
    stock_lst = stock_df.values.tolist()
    merge_stock_data=[]
        
    for i,row in enumerate(stock_lst):                 
        if i < 9:
            continue
        
        date = row[0].date().strftime('%Y%m%d')
        if date not in date_lst:
            continue
        
        sub_stock = stock_lst[i-9:i+1]
        pred_stock = stock_lst[i+1][-1] # d+1의 전날 대비 종가 변동성(change)
        result2=[]
            
        for j, row2 in enumerate(sub_stock):
            open, high, low, close, volume = row2[1:6]
            trading_value = close * volume
            result2 += [open, high, low, close, trading_value]
        
        
        result2 = ','.join(map(str,result2))
        ratio = (int(pred_stock >= 0.02)) # 2% 이상 상승이면, 1
        merge_stock_data.append([com_code, date, result2, ratio])

                
    return merge_stock_data

### multi_processing

In [40]:
result

[['000210', ['20210125', '20210204', '20210512']],
 ['004840', []],
 ['155660', []],
 ['078930', []],
 ['001250', ['20210427']],
 ['039570', []],
 ['011200',
  ['20210104',
   '20210105',
   '20210106',
   '20210107',
   '20210108',
   '20210111',
   '20210112',
   '20210128',
   '20210210',
   '20210215',
   '20210217',
   '20210219',
   '20210222',
   '20210223',
   '20210224',
   '20210225',
   '20210226',
   '20210302',
   '20210303',
   '20210304',
   '20210305',
   '20210308',
   '20210309',
   '20210310',
   '20210311',
   '20210312',
   '20210315',
   '20210316',
   '20210317',
   '20210318',
   '20210319',
   '20210322',
   '20210323',
   '20210324',
   '20210325',
   '20210326',
   '20210329',
   '20210330',
   '20210331',
   '20210401',
   '20210402',
   '20210405',
   '20210406',
   '20210407',
   '20210408',
   '20210409',
   '20210412',
   '20210413',
   '20210414',
   '20210415',
   '20210416',
   '20210419',
   '20210420',
   '20210421',
   '20210422',
   '20210423',
  

In [42]:
start_time=time.time()
com_code_lst = list(code2name_dic.keys())


# set_args=(result_dic,com_code_lst)

if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=20)
    result2 = pool.map( make_merge_stock_data, result)
    pool.close()
    pool.join()
    
print("---%s seconds ---" % (time.time() - start_time))

---10.80170226097107 seconds ---


In [43]:
OF=open('assignment3_multi_processing_test.txt','w')
for row in result2:
    for row2 in row:
        code, date, merge_sotck_data, ratio = row2
        OF.write('\t'.join(map(str,[code, date, merge_sotck_data, ratio]))+'\n')
OF.close()

# modeling

## 학습 데이터셋 (2018-01-01 ~ 2020-12-31)
## 시험 데이터셋 (2021-01-01 ~ 2021-06-31)

In [44]:
import numpy as np
start_time=time.time()

IF=open("assignment3_multi_processing.txt",'r')
lst_code_date=[]
trainX=[]
trainY=[]

'''train 데이터 준비'''
for line in IF:
    code, name, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    trainX.append(list(map(int, x.split(","))))
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

'''모델 학습'''
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(trainX, trainY)
print("---%s seconds ---" % (time.time() - start_time))

---9.03171420097351 seconds ---


In [45]:

import numpy as np
IF=open("assignment3_multi_processing_test.txt",'r')
lst_code_date=[]
testX=[]
testY=[]

'''test데이터 준비'''
for line in IF:
    code, name, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    testX.append(list(map(int, x.split(","))))
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)




predY = clf.predict(testX)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(testY, predY)
print(accuracy)

0.7609387430389817


In [50]:
predY=list(map(int, predY))
testY=list(map(int, testY))

In [60]:
labeled_1=0
correct_labeled_1=0
i=0
for pred, y in zip(predY,testY):
    if y==1:
        labeled_1+=1
    if y==1 and pred==1:
        correct_labeled_1+=1
    if y==0 and pred==1:
        i+=1
        

In [56]:
correct_labeled_1/labeled_1

0.024582967515364356

In [57]:
correct_labeled_1

28

In [62]:
print('라벨이 0인 데이터를 1이라고 예측한 개수:',i)
print('라벨이 1인 데이터 {}개 중에 라벨을 1이라고 맞춘 개수:{}\n성능은 {}'.format(labeled_1,correct_labeled_1,correct_labeled_1/labeled_1))

라벨이 0인 데이터를 1이라고 예측한 개수: 91
라벨이 1인 데이터 1139개 중에 라벨을 1이라고 맞춘 개수:28
성능은 0.024582967515364356
